In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext rpy2.ipython

In [12]:
import pandas as pd
from kaggle.house_prices import helpers
from kaggle.house_prices import missing

combined_dataset = helpers.load_data()
combined_dataset.shape

(2919, 82)

In [63]:
# testing replace_with_most_common
#
fixer = missing.FixMissing(combined_dataset)
fix_by_most_common = fixer.replace_with_most_common()

# numeric columns should not be affected
pd.testing.assert_frame_equal(
    fix_by_most_common.select_dtypes(include=[np.number]).sort_index(axis=1), 
    combined_dataset.select_dtypes(include=[np.number]).sort_index(axis=1)
)
 
# categ columns with `good` NAs should not be affected
pd.testing.assert_frame_equal(
    fix_by_most_common[missing.colums_with_valid_na].sort_index(axis=1), 
    combined_dataset[missing.colums_with_valid_na].sort_index(axis=1)
)

# there should not be NAs in processed columns
affected_columns = list(
    set(helpers.get_character_colnames(combined_dataset)) -  
    set(missing.colums_with_valid_na)
)
assert fix_by_most_common[affected_columns].isnull().values.sum() == 0